In [1]:
import librosa
import os
from numpy import random
import numpy
import os.path
import pydot

In [2]:
training_dir = 'audio_resources/'
training_files = [os.path.join(training_dir, f) for f in os.listdir(training_dir)]

In [3]:
def hash_func(vecs, projections):
    bools = numpy.dot(vecs, projections.T) > 0
    return [bool2int(bool_vec) for bool_vec in bools]

In [4]:
def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        if j: y += 1<<i
    return y

In [5]:
bool2int([False, True, False, True])

10

In [6]:
X = random.randn(10,100)
P = random.randn(3,100)
hash_func(X, P)

[6, 3, 4, 3, 7, 4, 4, 1, 2, 6]

In [7]:

class Table:
    
    def __init__(self, hash_size, dim):
        self.table = dict()
        self.hash_size = hash_size
        self.projections = random.randn(self.hash_size, dim)

    def add(self, vecs, label):
        entry = {'label': label}
        hashes = hash_func(vecs, self.projections)
        for h in hashes:
            if h in self.table:
                self.table[h].append(entry)
            else:
                self.table[h] = [entry]

    def query(self, vecs):
        hashes = hash_func(vecs, self.projections)
        results = list()
        for h in hashes:
            if h in self.table:
                results.extend(self.table[h])
        return results

In [8]:
class LSH:
    
    def __init__(self, dim):
        self.num_tables = 4
        self.hash_size = 8
        self.tables = list()
        for i in range(self.num_tables):
            self.tables.append(Table(self.hash_size, dim))
    
    def add(self, vecs, label):
        for table in self.tables:
            table.add(vecs, label)
    
    def query(self, vecs):
        results = list()
        for table in self.tables:
            results.extend(table.query(vecs))
        return results

    def describe(self):
        for table in self.tables:
            print(table.table)

In [9]:
class MusicSearch:
    
    def __init__(self, training_files):
        self.frame_size = 4096
        self.hop_size = 4000
        self.fv_size = 12
        self.lsh = LSH(self.fv_size)
        self.training_files = training_files
        self.num_features_in_file = dict()
        for f in self.training_files:
            self.num_features_in_file[f] = 0
                
    def train(self):
        for filepath in self.training_files:
            x, fs = librosa.load(filepath)
            features = librosa.feature.chroma_stft(x, fs, n_fft=self.frame_size, hop_length=self.hop_size).T
            self.lsh.add(features, filepath)
            self.num_features_in_file[filepath] += len(features)
                
    def query(self, filepath):
        x, fs = librosa.load(filepath)
        features = librosa.feature.chroma_stft(x, fs, n_fft=self.frame_size, hop_length=self.hop_size).T
        results = self.lsh.query(features)
        print ('num results', len(results))

        counts = dict()
        for r in results:
            if r['label'] in counts:
                counts[r['label']] += 1
            else:
                counts[r['label']] = 1
        for k in counts:
            counts[k] = float(counts[k])/self.num_features_in_file[k]
        return counts

In [10]:
ms = MusicSearch(training_files)
ms.train()

In [11]:
test_file = 'test/classical0.mp3'
results = ms.query(test_file)

num results 450434


In [12]:
for r in sorted(results, key=results.get, reverse=True):
    print (r, results[r])

audio_resources/metal3.mp3 52.6
audio_resources/hiphop0.mp3 52.18787878787879
audio_resources/disco1.mp3 52.10778443113772
audio_resources/blues2.mp3 51.224242424242426
audio_resources/metal4.mp3 50.78181818181818
audio_resources/disco5.mp3 49.1497005988024
audio_resources/rock1.mp3 47.59036144578313
audio_resources/country4.mp3 47.49090909090909
audio_resources/hiphop6.mp3 47.397590361445786
audio_resources/disco0.mp3 47.30120481927711
audio_resources/disco2.mp3 46.63855421686747
audio_resources/disco6.mp3 45.99393939393939
audio_resources/classical2.mp3 45.945454545454545
audio_resources/reggae2.mp3 45.81818181818182
audio_resources/metal5.mp3 45.59036144578313
audio_resources/metal2.mp3 45.30909090909091
audio_resources/disco4.mp3 45.15060240963855
audio_resources/rock2.mp3 45.024096385542165
audio_resources/hiphop5.mp3 44.72121212121212
audio_resources/reggae3.mp3 44.64242424242424
audio_resources/reggae4.mp3 44.39393939393939
audio_resources/rock0.mp3 44.03030303030303
audio_resou